In [1]:
from vllm import LLM, SamplingParams
from datasets import load_dataset
from tqdm.auto import tqdm
import gc

In [3]:
def generate_answers(model_id, dataset_name):
    dataset = load_dataset(dataset_name, split="test")

    def format(sample):
        return """Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n###
        Instruction:\n{}\n\n### Response:\n""".format(sample["instruction"])
    
    dataset = dataset.map(lambda sample: {"prompt": format(sample)})
    llm = LLM(model=model_id, max_model_len=4096)
    sampling_params = SamplingParams(temperature=0.8, top_p=0.95, min_p=0.05, max_tokens=4096)
    outputs = llm.generate(dataset["prompt"], sampling_params)
    answers = [output.outputs[0].text for output in outputs]
    dataset = dataset.add_column("answers", answers)
    print(f"Uploading results for {model_id}")
    dataset.push_to_hub(f"olawaleibrahim/{model_id.split('/')[-1]}-results")
    gc.collect()
    return dataset

In [2]:
model_ids = [
    "olawaleibrahim/DRA-3.1-8B",
    "olawaleibrahim/DRA-3.1-8B-DPO",
    "meta-llama/Meta-Llama-3.1-8B-Instruct"
]

In [4]:
for model_id in model_ids:
    generate_answers(model_id, "olawaleibrahim/professionaldocuments")

INFO 01-03 22:46:32 config.py:510] This model supports multiple tasks: {'classify', 'embed', 'score', 'reward', 'generate'}. Defaulting to 'generate'.
INFO 01-03 22:46:32 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='olawaleibrahim/DRA-3.1-8B', speculative_config=None, tokenizer='olawaleibrahim/DRA-3.1-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=o

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 5.80 GiB of which 86.88 MiB is free. Including non-PyTorch memory, this process has 5.70 GiB memory in use. Of the allocated memory 5.56 GiB is allocated by PyTorch, and 17.83 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [61]:
from datasets import load_dataset

# Load the dataset
# Replace 'dataset_name' with the name of the dataset you want to download
dataset = load_dataset("olawaleibrahim/DRA-3.1-8B-results")

In [62]:
df = pd.DataFrame(dataset)
df["style_score"] = df["test"].apply(lambda x: df["test"].values.tolist()[0]["style"])
df.iloc[0].values[0]["style"]

2

In [63]:
style = 0
accuracy = 0

values = df["test"].values
for i in range(df.shape[0]):
    if values[i]["style"]:
        style += int(values[i]["style"])
        accuracy += int(values[i]["accuracy"])
    else:
        print("None")

In [64]:
accuracy, style

(360, 304)

In [60]:
accuracy, style #DPO

(349, 303)

In [71]:
303/ (154*3)

0.6558441558441559